# Group Transforms and "Unwrapped" GroupBy 

In [2]:
import pandas as pd, numpy as np

Like apply method there is another built-in method called transform, which is similar to apply but impose more constraints on the kind of 
1. it can produce a scalr value to be broadcast to the shape of the group
2. It can produce an object of the same shape as the input group
3. It must not mutate its input

In [3]:
df = pd.DataFrame({
    'key': ['a', 'b', 'c'] *4,
    'value': np.arange(12.)
                  })
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [5]:
g = df.groupby('key')
g['value'].mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [7]:
g.transform(lambda x:x.mean())

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [8]:
g.transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [14]:
g.apply(lambda x:x.mean())

,value
key,
a,4.5
b,5.5
c,6.5


In [15]:
g.apply(lambda x:x * 2)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [16]:
g.transform(lambda x: x *2)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [17]:
g.transform(lambda x:x.rank(ascending=False))

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [18]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [19]:
g.transform(normalize)

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


In [25]:
normalized = (df['value'] - g['value'].transform('mean')) /g['value'].transform('std')
normalized # unwrapped group operations may invlove multiple group aggregations, overall benefit of vectorized operations often outwighs this.

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# Group Time Resampling

In [38]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({
                 'time': times,
                 'value': np.arange(N)
                 })
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [39]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [40]:
df2 = pd.DataFrame({'time': times.repeat(3),
                   'key' : np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)
                   })
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [44]:
time_key = pd.Grouper(freq='5min')
resampled = (df2.set_index('time')
            .groupby(['key', time_key])
            .sum() 
            )
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [45]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0
